In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np
from tqdm import tqdm

In [2]:
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

In [3]:
def create_features(rel_df, pos):
    if pos ==  "wr":
        for col in rel_df.columns[4:]:
            if "FPTS_TG" not in col and rel_df[col].dtype !="O":
                new_c = rel_df[col]/rel_df["AVG"]

                new_name = col+"_ADP"

                rel_df[new_name] = new_c
                
    if pos == "rb" or pos ==  "wr" or pos == "qb" or pos == "te":
        for col in rel_df.columns[4:]:
            if "FPTS" not in col and "_ADP" not in col:
                if "_avg" in col:
                    new_c = rel_df["FPTS/G_MISC_avg"]*rel_df[col]
                elif "_ewm" in col:
                    new_c = rel_df["FPTS/G_MISC_ewm"]*rel_df[col]
                
                new_name = col+"_FP"

                rel_df[new_name] = new_c
    return rel_df

In [4]:
def train(pos):
    df = pd.read_csv(f"processed data/{pos}_proc_data.csv")
    df = df[df["Rookie"] == 0]
    df = df.drop(["Rookie"], axis=1)
    #return df
    df_feats = create_features(df, pos)
    
    
    with open(f"model features/{pos}_feats", "rb") as fp:   # Unpickling
        model_cols = pickle.load(fp)
        
    #print(model_cols)
    model_df = df_feats[model_cols].dropna()
    
    X = model_df.drop(["FPTS_TG", "Season"], axis=1)
    y = model_df[["FPTS_TG"]]

    if pos == "wr":
        rf_opt = RandomForestRegressor(max_depth=5, max_features="sqrt", min_samples_split = 10, n_estimators=400, random_state=0)
    elif pos == "rb":
        rf_opt = RandomForestRegressor(max_depth=10, max_features="sqrt", min_samples_split = 5, n_estimators=400, random_state=0)
    elif pos == "qb":
        rf_opt = RandomForestRegressor(max_depth=20, max_features="sqrt", min_samples_split = 5, n_estimators=700, random_state=0)
    elif pos == "te":
        rf_opt = RandomForestRegressor(max_depth=20, max_features="sqrt", min_samples_split = 10, n_estimators=300, random_state=0)
    sc = preprocessing.StandardScaler()
    #X = sc.fit_transform(X)
    
    
    rf_opt.fit(X, y.values.ravel())
    
    return rf_opt, sc#[model_df.Season == 2022]

In [5]:
def predict_ns(pos, model, sc):
    df = pd.read_csv(f"processed data/{pos}_ns_proc.csv")
    
    df["Position"] = [pos]*len(df)
    df = df[df["Rookie"] == 0]
    df = df.drop(["Rookie"], axis=1)
    df = df[~df["YDS_RUSHING/G_avg"].isna()]
    #return df.columns[4:]
    df_feats = create_features(df, pos)
    
    
    with open(f"model features/{pos}_feats", "rb") as fp:   # Unpickling
        model_cols = pickle.load(fp)
    #return model_cols
    model_df = df_feats[model_cols]
    
    X = model_df.drop(["FPTS_TG", "Season"], axis=1)
    X_sc = X#sc.transform(X)
    
    pred = model.predict(X_sc.fillna(0))
    
    df_feats["FPTS_TG_pred"] = pred
    
    df_board = df_feats[["pid", "Player", "Season", "AVG", "FPTS_TG_pred", "STD", "Position"]]
    return df_board

In [8]:
model, scaler = train('wr')
    
pred_board = predict_ns('wr', model, scaler)

In [9]:
pred_board.sort_values("FPTS_TG_pred", ascending=0).head(30)#d#[["Player", "Season", "FPTS_TG", "FPTS/G_MISC_avg", "FPTS_MISC_avg", "FPTS_MISC_ewm"]]

,pid,Player,Season,AVG,FPTS_TG_pred,STD,Position
3,15802,Tyreek Hill MIA (10),2023,7.3,16.316038,0.577350,wr
4,13981,Stefon Diggs BUF (13),2023,11.3,16.298992,1.154701,wr
0,19236,Justin Jefferson MIN (13),2023,1.3,16.161827,0.577350,wr
1,19788,Ja'Marr Chase CIN (7),2023,3.0,15.978501,0.000000,wr
2,16433,Cooper Kupp LAR (10),2023,5.3,15.930688,0.577350,wr
5,18218,A.J. Brown PHI (10),2023,13.7,14.560298,3.055050,wr
7,12123,Davante Adams LV (13),2023,16.0,14.322796,1.000000,wr
6,19202,CeeDee Lamb DAL (7),2023,13.7,13.330366,2.516611,wr
8,23072,Garrett Wilson NYJ (7),2023,19.0,13.173302,1.000000,wr
10,19790,Jaylen Waddle MIA (10),2023,23.3,12.971355,2.081666,wr


In [84]:
pos = ["wr", "rb", "te", "qb"]

full_board = pd.DataFrame()

for p in tqdm(pos):
    model, scaler = train(p)
    
    pred_board = predict_ns(p, model, scaler)
    
    full_board = pd.concat([full_board, pred_board])

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

['Season', 'AVG', 'FPTS_TG', 'FPTS/G_MISC_avg', 'REC_RECEIVING/G_avg', 'TGT_RECEIVING/G_avg', 'YDS_RECEIVING/G_avg', 'TD_RECEIVING/G_avg', 'FPTS/G_MISC_ewm', 'REC_RECEIVING/G_ewm', 'TGT_RECEIVING/G_ewm', 'YDS_RECEIVING/G_ewm', 'TD_RECEIVING/G_ewm', 'YDS/REC_RECEIVING_avg_ADP', 'YDS/TGT_RECEIVING_avg_ADP', 'REC/TGT_RECEIVING_avg_ADP', 'YDS/REC_RECEIVING_ewm_ADP', 'YDS/TGT_RECEIVING_ewm_ADP', 'REC/TGT_RECEIVING_ewm_ADP', 'REC_RECEIVING/G_avg_FP', 'TGT_RECEIVING/G_avg_FP', 'YDS_RECEIVING/G_avg_FP', 'YDS/TGT_RECEIVING_avg_FP', 'REC/TGT_RECEIVING_avg_FP', 'REC_RECEIVING/G_ewm_FP', 'TGT_RECEIVING/G_ewm_FP', 'YDS_RECEIVING/G_ewm_FP', 'YDS/REC_RECEIVING_ewm_FP', 'YDS/TGT_RECEIVING_ewm_FP', 'REC/TGT_RECEIVING_ewm_FP', 'Career_Years_FP']


 25%|█████████████████████                                                               | 1/4 [00:02<00:07,  2.37s/it]

['Season', 'AVG', 'FPTS_TG', 'FPTS/G_MISC_avg', 'YDS_RUSHING/G_avg', 'FPTS/G_MISC_ewm', 'YDS_RUSHING/G_ewm', 'TD_RUSHING/G_ewm', 'ATT_RUSHING/G_avg_FP', 'YDS_RUSHING/G_avg_FP', 'TD_RUSHING/G_avg_FP', 'REC_RECEIVING/G_avg_FP', 'TGT_RECEIVING/G_avg_FP', 'YDS_RECEIVING/G_avg_FP', 'YDS/ATT_RUSHING_avg_FP', 'ATT_RUSHING/G_ewm_FP', 'YDS_RUSHING/G_ewm_FP', 'TD_RUSHING/G_ewm_FP', 'REC_RECEIVING/G_ewm_FP', 'TGT_RECEIVING/G_ewm_FP', 'YDS_RECEIVING/G_ewm_FP', 'YDS/ATT_RUSHING_ewm_FP', 'Career_Years_FP']


 50%|██████████████████████████████████████████                                          | 2/4 [00:04<00:04,  2.41s/it]

['Season', 'AVG', 'FPTS_TG', 'FPTS/G_MISC_avg', 'REC_RECEIVING/G_avg', 'YDS_RECEIVING/G_avg', 'FPTS/G_MISC_ewm', 'REC_RECEIVING/G_ewm', 'TGT_RECEIVING/G_ewm', 'YDS_RECEIVING/G_ewm', 'REC_RECEIVING/G_avg_FP', 'TGT_RECEIVING/G_avg_FP', 'YDS_RECEIVING/G_avg_FP', 'REC_RECEIVING/G_ewm_FP', 'TGT_RECEIVING/G_ewm_FP', 'YDS_RECEIVING/G_ewm_FP', 'TD_RECEIVING/G_ewm_FP']


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:06<00:01,  1.86s/it]

['Season', 'AVG', 'FPTS_TG', 'FPTS/G_MISC_avg', 'YDS_PASSING/G_avg', 'TD_PASSING/G_avg', 'FPTS/G_MISC_ewm', 'YDS_PASSING/G_ewm', 'TD_PASSING/G_ewm', 'Y/A_PASSING_avg_FP', 'CMP_PASSING/G_avg_FP', 'ATT_PASSING/G_avg_FP', 'YDS_PASSING/G_avg_FP', 'TD_PASSING/G_avg_FP', 'Y/A_PASSING_ewm_FP', 'CMP_PASSING/G_ewm_FP', 'ATT_PASSING/G_ewm_FP', 'YDS_PASSING/G_ewm_FP', 'TD_PASSING/G_ewm_FP']


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.23s/it]


In [85]:
#pred_board.sort_values("FPTS_TG_pred", ascending=False).head(20)

In [86]:
#pred_board.sort_values("FPTS_TG_pred", ascending=False).head(20)

In [87]:
full_board = full_board.sort_values("AVG")

In [88]:
full_board.sort_values("FPTS_TG_pred", ascending=False).head(30)

,pid,Player,Season,AVG,FPTS_TG_pred,STD,Position
1,17298,Josh Allen BUF (13),2023,20.0,21.682469,4.358899,qb
0,16413,Patrick Mahomes II KC (10),2023,15.0,20.733982,1.000000,qb
2,19275,Jalen Hurts PHI (10),2023,22.7,20.411460,1.154701,qb
6,19781,Justin Fields CHI (13),2023,47.0,20.300229,2.000000,qb
3,17233,Lamar Jackson BAL (13),2023,34.3,20.153940,3.055050,qb
4,19196,Joe Burrow CIN (7),2023,35.0,20.153888,3.605551,qb
5,18635,Justin Herbert LAC (5),2023,46.0,19.279383,5.000000,qb
8,15600,Dak Prescott DAL (7),2023,77.7,19.254327,9.451631,qb
9,16398,Deshaun Watson CLE (5),2023,82.7,18.362930,4.509250,qb
12,11177,Kirk Cousins MIN (13),2023,107.0,18.292942,4.358899,qb


In [89]:
full_board.to_csv("board.csv", index=False)

In [37]:
import scipy.stats

In [89]:
1- scipy.stats.norm(11.3, 1.51).cdf(4)

0.9999993323853829